In [1]:
from tqdm import tqdm
import tensorflow as tf
from tensorflow.keras.layers import Dense, Conv2D, MaxPooling2D, Flatten, Dropout, BatchNormalization

In [2]:
cifar10 = tf.keras.datasets.cifar10

(X_train, Y_train), (X_test, Y_test) = cifar10.load_data()
X_train = X_train / 255.0
X_test = X_test / 255.0

170498071/170498071 [==============================] - 2s 0us/step


In [3]:
X_train[0].shape

(32, 32, 3)

In [4]:
train_dataset = tf.data.Dataset.from_tensor_slices((X_train, Y_train)).shuffle(1000).batch(32)
test_dataset = tf.data.Dataset.from_tensor_slices((X_test, Y_test)).batch(32)

In [23]:
class MyModel(tf.keras.Model):
  def __init__(self):
    super().__init__()
    self.conv1 = Conv2D(32, (3,3), activation = "relu")
    self.b1 = BatchNormalization()
    self.conv2 = Conv2D(32, (3, 3), activation = "relu")
    self.b2 = BatchNormalization()
    self.maxpool1 = MaxPooling2D(2,2)
    self.dropout1 = Dropout(0.2)

    self.conv3 = Conv2D(64,(3, 3), activation='relu', kernel_initializer ='he_uniform', padding='same')
    self.b3 = BatchNormalization()
    self.conv4 = Conv2D(64,(3, 3), activation='relu', kernel_initializer='he_uniform', padding='same')
    self.b4 = BatchNormalization()
    self.maxpool2 = MaxPooling2D(2, 2)
    self.dropout2 = Dropout(0.3)

    self.conv5 = Conv2D(128,(3, 3), activation='relu', kernel_initializer='he_uniform', padding='same')
    self.b5 = BatchNormalization()
    self.conv6 = Conv2D(128,(3, 3), activation='relu', kernel_initializer='he_uniform', padding='same')
    self.b6 = BatchNormalization()
    self.maxpool3 = MaxPooling2D(2, 2)
    self.dropout3 = Dropout(0.4)


    self.f = Flatten()
    self.d1 = Dense(128,activation = 'relu')
    self.b7 = BatchNormalization()
    self.dropout4 = Dropout(0.5)
    self.d2 = Dense(10,activation='softmax')

  def call(self, x):
    x = self.conv1(x)
    x = self.b1(x)
    x = self.conv2(x)
    x = self.b2(x)
    x = self.maxpool1(x)
    x = self.dropout1(x)
    x = self.conv3(x)
    x = self.b3(x)
    x = self.conv4(x)
    x = self.b4(x)
    x = self.maxpool2(x)
    x = self.dropout2(x)
    x = self.conv5(x)
    x = self.b5(x)
    x = self.conv6(x)
    x = self.b6(x)
    x = self.maxpool3(x)
    x = self.dropout3(x)
    x = self.f(x)
    x = self.d1(x)
    x = self.b7(x)
    x = self.dropout4(x)
    x = self.d2(x)
    return x

model = MyModel()


In [24]:
loss_function = tf.keras.losses.SparseCategoricalCrossentropy(from_logits = True)
optimizer = tf.keras.optimizers.Adam()

In [25]:
train_loss = tf.keras.metrics.Mean()
test_loss = tf.keras.metrics.Mean()

train_accuracy = tf.keras.metrics.SparseCategoricalAccuracy()
test_accuracy = tf.keras.metrics.SparseCategoricalAccuracy()

In [26]:
@tf.function
def train(images, labels):
  with tf.GradientTape() as tape:
      pred = model(images)
      loss = loss_function(labels, pred)

  gradients = tape.gradient(loss, model.trainable_variables)
  optimizer.apply_gradients(zip(gradients, model.trainable_variables))
  train_loss(loss)
  train_accuracy(labels, pred)

In [27]:
@tf.function
def test(images, labels):
  pred = model(images)
  loss = loss_function(labels, pred)
  test_loss(loss)
  test_accuracy(labels, pred)


In [28]:
epochs = 10

for epoch in tqdm(range(epochs)):
  train_loss.reset_states()
  train_accuracy.reset_states()
  test_loss.reset_states()
  test_accuracy.reset_states()

  #train
  for images, labels in train_dataset:
    train(images, labels)


  #test
  for images,labels in test_dataset:
    test(images, labels)

  print(f"Epoch : {epoch+1} , Train Loss : {train_loss.result()} , Train Acuuracy : {train_accuracy.result()} , Test Loss : {test_loss.result()} , Test Accuracy : {test_accuracy.result()}")

  0%|          | 0/10 [00:00<?, ?it/s]/usr/local/lib/python3.10/dist-packages/keras/src/backend.py:5727: UserWarning: "`sparse_categorical_crossentropy` received `from_logits=True`, but the `output` argument was produced by a Softmax activation and thus does not represent logits. Was this intended?
  output, from_logits = _get_logits(
 10%|█         | 1/10 [00:18<02:43, 18.22s/it]

Epoch : 1 , Train Loss : 1.3395527601242065 , Train Acuuracy : 0.514680027961731 , Test Loss : 0.983324408531189 , Test Accuracy : 0.6539000272750854


 20%|██        | 2/10 [00:27<01:43, 12.91s/it]

Epoch : 2 , Train Loss : 0.8087024688720703 , Train Acuuracy : 0.7171000242233276 , Test Loss : 0.8205610513687134 , Test Accuracy : 0.7186999917030334


 30%|███       | 3/10 [00:36<01:17, 11.13s/it]

...

Epoch : 8 , Train Loss : 0.2082156389951706 , Train Acuuracy : 0.9261999726295471 , Test Loss : 1.0179146528244019 , Test Accuracy : 0.7674999833106995


 90%|█████████ | 9/10 [01:29<00:09,  9.03s/it]

Epoch : 9 , Train Loss : 0.17695429921150208 , Train Acuuracy : 0.9378600120544434 , Test Loss : 1.038375973701477 , Test Accuracy : 0.76910001039505


100%|██████████| 10/10 [01:40<00:00, 10.00s/it]

Epoch : 10 , Train Loss : 0.161626935005188 , Train Acuuracy : 0.9437599778175354 , Test Loss : 1.1030232906341553 , Test Accuracy : 0.7663999795913696


In [ ]:
import cv2
import numpy as np

labels = ["airplane", "cars", "birds", "cats", "deer", "dogs", "frogs", "horses", "ships","trucks"]

image = cv2.imread("/content/airplane.jpeg")
image = cv2.resize(image, (32, 32))
image = image[tf.newaxis, ...]
image = image.astype("float32")
image.shape


In [36]:
pred = model(image)
result = np.argmax(pred)
label = labels[result]
label


'airplane'

In [37]:
model.save("cifar10_model")

In [38]:
model = tf.keras.models.load_model("cifar10_model")